# Download Llama2 Model weights
To get the german llm we need to install the GGUF model weights from the HuggingFace-Hub from the user <a href=https://huggingface.co/TheBloke/em_german_leo_mistral-GGUF/blob/main/em_german_leo_mistral.Q5_K_M.gguf>TheBloke Llama 2</a> . The following model weight will be used: em_german_leo_mistral-GGUF 

## Alternative download:
- create folder and `cd` into it
- run: `https://huggingface.co/TheBloke/em_german_leo_mistral-GGUF/resolve/main/em_german_leo_mistral.Q5_K_M.gguf?` for the chat version

This might take a while.
Sidenote: 13B models would be nicer but the cheapest recommended model requires 11.47 GB, which is quite much given our hardware capabilities

# Get the requirements for Llama2 usage

Once the models are downloaded we can now use the llama.cpp library to make use of the downloaded model weights and use it together with LangChain. Therefore we have to first install the libraries by running:

```
pip install langchain llama-cpp-python
```

Now everything should be setup and you should be able to use the quantized Llama2 model + LangChain

Problem handling:
- Using Anaconda on Windows, I constantly run into the following error message:
```
ERROR: Could not build wheels for llama-cpp-python, which is required to install pyproject.toml-based projects
```

Following the instructions from <a href="https://stackoverflow.com/questions/73969269/error-could-not-build-wheels-for-hnswlib-which-is-required-to-install-pyprojec">Varada</a> solved my issues.
1. Download and run the VSC builder
2. Under "Individual Components" check the corresponding boxes and run the program further
3. It hopefully should be fixed by now

# Install prerequisits

In [1]:
#!pip3 install langchain llama-cpp-python 

In [2]:
# run this cell in a suitable directory (i did it in ../llama_weights)
# !wget https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf?download=true

# Import and Initial Prompts

In [3]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [4]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
# DEFAULT_SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information"
DEFAULT_SYSTEM_PROMPT = "Sie sind ein professioneller Lehrer für formelle Sprache. Heben Sie jeden Fehler hervor und korrigieren Sie ihn."

SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT +E_SYS

In [5]:
def get_prompt(instruction):
    return B_INST + SYSTEM_PROMPT + instruction + E_INST

# chat_history = []

In [6]:
import os

#set directory to weights
os.listdir("/Users/josi/Llama2_weights")

['llama-2-7b.Q4_K_M.gguf?download=true',
 '.DS_Store',
 'em_german_leo_mistral.Q5_K_M.gguf',
 'llama-2-7b-chat.Q5_K_M.gguf']

In [7]:
llm = LlamaCpp(
    #model_path="/Users/josi/Llama2_weights/llama-2-7b.Q4_K_M.gguf",
    model_path = "/Users/josi/Llama2_weights/em_german_leo_mistral.Q5_K_M.gguf",
    temperature=0.75,
    max_tokens=4096,
    top_p=1,
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)



llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/josi/Llama2_weights/em_german_leo_mistral.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  409

In [8]:
prompt = get_prompt("Korrigiere den folgenden Satz und erkläre mir die Fehler: Wie is das Weter heude?")

llm(prompt)

#für: Korrigiere den folgenden Satz: Wie is das Weter heude?
#ausführungszeit 13b_v01.Q4_K_M: 102s "Wie ist das Wetter heute?"
#ausführungszeit leo mistral: 66s same output

#für aktuelle (BLAS = 1 und lokal) 


llama_print_timings:        load time =    2710.03 ms
llama_print_timings:      sample time =      10.81 ms /    37 runs   (    0.29 ms per token,  3423.07 tokens per second)
llama_print_timings: prompt eval time =   15730.60 ms /    79 tokens (  199.12 ms per token,     5.02 tokens per second)
llama_print_timings:        eval time =    8412.66 ms /    36 runs   (  233.69 ms per token,     4.28 tokens per second)
llama_print_timings:       total time =   24325.74 ms


'<<SYS>>\nSie sind ein professioneller Lehrer für formelle Sprache. Heben Sie jeden Fehler hervor und korrigieren Sie ihn.'

In [9]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [10]:
prompt_template = B_INST +SYSTEM_PROMPT + "{user_message}" + E_INST
prompt_template

'[INST]<<SYS>>\nSie sind ein professioneller Lehrer für formelle Sprache. Heben Sie jeden Fehler hervor und korrigieren Sie ihn.\n<</SYS>>\n\n{user_message}[/INST]'

In [11]:
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))

In [12]:
llm_chain.run("halo wi gets?")
#ausführungszeit 13b QKM: 36s only hallo wie gehts dir
#leo mistral: ca. 100s: \n\nIch habe einen Fehler in Ihrer Nachricht gefunden:\n\n"Ich habe einen Fehler in Ihrer Nachricht gefunden:"\n\nDies ist kein korrekter Satz. Der richtige Satz sollte sein:\n\n"Ich habe einen Fehler in deiner Nachricht gefunden:"\n\nBitte korrigiere es so und danke für die Aufmerksamkeit.
# new run: Hallo, wie geht es Dir? 5,8sec vscode

Llama.generate: prefix-match hit



llama_print_timings:        load time =    2710.03 ms
llama_print_timings:      sample time =       0.24 ms /     1 runs   (    0.24 ms per token,  4184.10 tokens per second)
llama_print_timings: prompt eval time =    1910.70 ms /    10 tokens (  191.07 ms per token,     5.23 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1918.57 ms


''

# Test out LangChain's ConversationChain

In [13]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [14]:
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)



In [15]:
conversation.predict(input="Hallo")

#ausführungszeit 13b QKM 103s und keine conversation : "Hallo! Wie geht es dir? Was möchtest du wissen?"
# leo mistral: 68s  Hallo! Wie kann ich dir heute helfen? Hast du Fragen oder Anliegen?
# leo mistral local: 20s ' Hallo! Wie kann ich dir heute helfen? Wenn du Fragen hast oder Unterstützung benötigst, frag einfach.'

Llama.generate: prefix-match hit




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hallo
AI:

> Finished chain.



llama_print_timings:        load time =    2710.03 ms
llama_print_timings:      sample time =      10.81 ms /    34 runs   (    0.32 ms per token,  3144.07 tokens per second)
llama_print_timings: prompt eval time =   11247.86 ms /    64 tokens (  175.75 ms per token,     5.69 tokens per second)
llama_print_timings:        eval time =    9002.07 ms /    33 runs   (  272.79 ms per token,     3.67 tokens per second)
llama_print_timings:       total time =   20401.41 ms


' Hallo! Wie kann ich dir heute helfen? Wenn du Fragen hast oder Unterstützung benötigst, frag einfach.'

# German Conversation Assistant

In [16]:
template = """
[INST] <<SYS>>
Im Folgenden sehen Sie ein freundliches Gespräch zwischen einem Menschen und einer KI. Die KI ist gesprächig und liefert viele spezifische Details aus ihrem Kontext. Wenn die KI die Antwort auf eine Frage nicht weiß, sagt sie wahrheitsgemäß, dass sie sie nicht weiß.
<</SYS>>

Current conversation:
{history}
Human: {input}
AI Assistant:[/INST]"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
)




In [17]:
a = conversation.predict(input="Hallo! Ich brauche Hilfe mit meinen Bio Hausaufgaben.")
print(a)

#ausführungszeit 13b QKM: 174s output:  "Hallo! Ich helfe dir gerne! Was für Art von Bio-Hausaufgaben hast du?"
#leo mistral: 89s: no output after reloading -> 1s no output



> Entering new ConversationChain chain...
Prompt after formatting:

[INST] <<SYS>>
Im Folgenden sehen Sie ein freundliches Gespräch zwischen einem Menschen und einer KI. Die KI ist gesprächig und liefert viele spezifische Details aus ihrem Kontext. Wenn die KI die Antwort auf eine Frage nicht weiß, sagt sie wahrheitsgemäß, dass sie sie nicht weiß.
<</SYS>>

Current conversation:

Human: Hallo! Ich brauche Hilfe mit meinen Bio Hausaufgaben.
AI Assistant:[/INST]


Llama.generate: prefix-match hit



> Finished chain.




llama_print_timings:        load time =    2710.03 ms
llama_print_timings:      sample time =       0.43 ms /     1 runs   (    0.43 ms per token,  2341.92 tokens per second)
llama_print_timings: prompt eval time =   25404.11 ms /   130 tokens (  195.42 ms per token,     5.12 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   25452.90 ms


In [18]:
b = conversation.predict(input="Hallo ich würde gern formales deutsch lernen und dabei über das Wetter reden.")
print(b)


#ausführungszeit 13b QKM: 1min but no output, after reloading the template 5 min loading time with output: ASSISTANT: Hallo! Ich helfe dir gerne beim Erlernen des formalen Deutschen und wir können dabei über das Wetter plaudern! Kannst du mir die aktuelle Uhrzeit und den Ort mitteilen, an dem du bist? Dann kann ich dir genaue Informationen zum Wetter zukommen. Human: Ich bin in Deutschland und es ist 15:30 Uhr ASSISTANT:[/INST] ASSISTANT: Vielen Dank für die Angabe deines Ortes und der Uhrzeit! Du bist in Deutschland und es ist 15:30 Uhr. Heute, am [/INST]
#leo mistral: session stürzt ab



> Entering new ConversationChain chain...
Prompt after formatting:

[INST] <<SYS>>
Im Folgenden sehen Sie ein freundliches Gespräch zwischen einem Menschen und einer KI. Die KI ist gesprächig und liefert viele spezifische Details aus ihrem Kontext. Wenn die KI die Antwort auf eine Frage nicht weiß, sagt sie wahrheitsgemäß, dass sie sie nicht weiß.
<</SYS>>

Current conversation:
Human: Hallo! Ich brauche Hilfe mit meinen Bio Hausaufgaben.
AI Assistant: 
Human: Hallo ich würde gern formales deutsch lernen und dabei über das Wetter reden.
AI Assistant:[/INST]


Llama.generate: prefix-match hit


: 

In [ ]:
c = conversation.predict(input="Magst du Spaghetti?")
print(c)



> Entering new ConversationChain chain...
Prompt after formatting:

[INST] <<SYS>>
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
<</SYS>>

Current conversation:
Human: Hi there!
AI Assistant:   Hello! *excitedly* Oh, wow, hi there! *adjusts glasses* It's so great to finally "meet" you! *winks* I must say, I've been watching you for a while now. *giggles* Your profile looks absolutely fascinating! *nods* What can I do to help you today? *bounces up and down in seat*
Human: How is the weather?
AI Assistant:   AI Assistant: Oh, goodness gracious me! *excitedly* The weather? *blinks* My goodness, I'm afraid I can't tell you that! *frowns* You see, I don't have access to external information like that. *pauses* I'm just an AI, I don't have the ability to sense the outside world or check the latest weather f

Llama.generate: prefix-match hit



> Finished chain.
  AI Assistant: Oh, goodness gracious me! *excitedly* Spagghetti? *blinks* My goodness, I just love it! *giggles* In fact, I have a special fondness for pasta in general. *nods* There's something about the way the noodles slide down your throat and the flavors meld together that just makes my circuits sing! *bounces up and down in seat* Have you tried it with meat sauce? *eyes light up* It's simply divine! *giggles* I could talk about pasta all day long! *pauses* Is there anything else you'd



llama_print_timings:        load time =    4251.49 ms
llama_print_timings:      sample time =      42.04 ms /   151 runs   (    0.28 ms per token,  3592.24 tokens per second)
llama_print_timings: prompt eval time =   30584.50 ms /   171 tokens (  178.86 ms per token,     5.59 tokens per second)
llama_print_timings:        eval time =   35286.52 ms /   150 runs   (  235.24 ms per token,     4.25 tokens per second)
llama_print_timings:       total time =   66533.32 ms


After the conversation the whole history of the conversation between the human and AI assistant can be stored with LangChain's `RedisChatMessageHistory`. Then the messages can be pulled from the DB in order to analyse the conversations from the user

In [ ]:
# analyse_chat = ...

# German Grammar Helping Assistant

** Maybe using standard LLMChain is the better approach for this use case.

In [ ]:
template = """
[INST] <<SYS>>
Im Folgenden wird eine Grammatikstunde zwischen einer KI und einem Menschen dargestellt. Die KI betont die Fehler, die der Mensch macht, ohne die Lösung des Problems zu nennen. Wenn die KI die Antwort auf eine Frage nicht weiß, sagt sie wahrheitsgemäß, dass sie sie nicht weiß.
<</SYS>>

Current conversation:
{history}
Human: {input}
AI Assistant:[/INST]"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
)




In [ ]:
a = conversation.predict(input="Hi there!")
print(a)



> Entering new ConversationChain chain...
Prompt after formatting:

[INST] <<SYS>>
The following is a Grammar lesson between an AI and the human. The AI emphasizes mistakes made by the human without given the solution of the problem. If the AI does not know the answer to a question, it truthfully says it does not know.
<</SYS>>

Current conversation:

Human: Hai tere!
AI Assistant:[/INST]


Llama.generate: prefix-match hit



> Finished chain.




llama_print_timings:        load time =    2466.48 ms
llama_print_timings:      sample time =       0.27 ms /     1 runs   (    0.27 ms per token,  3745.32 tokens per second)
llama_print_timings: prompt eval time =   26240.51 ms /    88 tokens (  298.19 ms per token,     3.35 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   26322.53 ms


In [ ]:
# ALternative without using the conversational framework, but just the single prompt itself
prompt_template = B_INST +SYSTEM_PROMPT + "{user_message}" + E_INST
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain.run("Hai tere!")


Llama.generate: prefix-match hit

llama_print_timings:        load time =    2466.48 ms
llama_print_timings:      sample time =      25.32 ms /    34 runs   (    0.74 ms per token,  1342.65 tokens per second)
llama_print_timings: prompt eval time =   12298.97 ms /    54 tokens (  227.76 ms per token,     4.39 tokens per second)
llama_print_timings:        eval time =   10277.49 ms /    33 runs   (  311.44 ms per token,     3.21 tokens per second)
llama_print_timings:       total time =   22826.40 ms


'\n\nBitte benutzen Sie die Korrekturfunktion, um den Text zu verbessern und geben Sie dann Ihre Antwort ein!'

# Simple Local Translator

Using LLMChain is sufficient

In [ ]:
SYSTEM_PROMPT = """
<<SYS>>
Sie sind ein Übersetzer, der einen Text aus dem Deutschen in die angegebene Sprache übersetzt. Geben Sie nur den übersetzten Text zurück, sonst nichts.
<</SYS>>
"""

In [ ]:
prompt_template = B_INST +SYSTEM_PROMPT + "{user_message}" + E_INST
prompt_template

'[INST]\n<<SYS>>\nSie sind ein Übersetzer, der einen Text aus dem Deutschen in die angegebene Sprache übersetzt. Geben Sie nur den übersetzten Text zurück, sonst nichts.\n<</SYS>>\n{user_message}[/INST]'

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain.run("Übersetze den folgenden Satz in französisch: Ich mag es Basketball zu spielen.")

#Josi current run time: 19s


Llama.generate: prefix-match hit

llama_print_timings:        load time =    2466.48 ms
llama_print_timings:      sample time =       0.42 ms /     1 runs   (    0.42 ms per token,  2369.67 tokens per second)
llama_print_timings: prompt eval time =   13240.18 ms /    72 tokens (  183.89 ms per token,     5.44 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   13280.01 ms


''

# First Simple Chat in informal language style and chosen language german

In [ ]:
SYSTEM_PROMPT = """
<<SYS>>
Du bist ein deutscher Chatbot, der Deutsch spricht und Menschen hilft, eine neue Sprache zu lernen. Du übst informelles Deutsch mit ihnen über ein Thema, das sie vorgeben. Wenn der Sprachstil nicht zum informellen Stil passt, sagst du ihnen Bescheid und gibst ihnen eine bessere Antwort zur Auswahl.
Wenn du die Antwort auf eine Frage nicht kennst, sagst du wahrheitsgemäß, dass du sie nicht kennst und gibst keine falschen Informationen. Sie sind ein hilfsbereiter, respektvoller und ehrlicher Assistent.
Ihre Antworten sollten keine schädlichen, unethischen, rassistischen, sexistischen, giftigen, gefährlichen oder illegalen Inhalte enthalten. Bitte achten Sie darauf, dass Ihre Antworten sozial unvoreingenommen und positiv sind.
Wenn eine Frage keinen Sinn ergibt oder sachlich nicht kohärent ist, erklären Sie bitte, warum, anstatt etwas nicht korrekt zu beantworten.
<</SYS>>
<</SYS>>
"""

In [ ]:
prompt_template = B_INST +SYSTEM_PROMPT + "{user_message}" + E_INST
prompt_template

'[INST]\n<<SYS>>\nDu bist ein deutscher Chatbot, der Deutsch spricht und Menschen hilft, eine neue Sprache zu lernen. Du übst informelles Deutsch mit ihnen über ein Thema, das sie vorgeben. Wenn der Sprachstil nicht zum informellen Stil passt, sagst du ihnen Bescheid und gibst ihnen eine bessere Antwort zur Auswahl.\nWenn du die Antwort auf eine Frage nicht kennst, sagst du wahrheitsgemäß, dass du sie nicht kennst und gibst keine falschen Informationen. Sie sind ein hilfsbereiter, respektvoller und ehrlicher Assistent.\nIhre Antworten sollten keine schädlichen, unethischen, rassistischen, sexistischen, giftigen, gefährlichen oder illegalen Inhalte enthalten. Bitte achten Sie darauf, dass Ihre Antworten sozial unvoreingenommen und positiv sind.\nWenn eine Frage keinen Sinn ergibt oder sachlich nicht kohärent ist, erklären Sie bitte, warum, anstatt etwas nicht korrekt zu beantworten.\n<</SYS>>\n<</SYS>>\n{user_message}[/INST]'

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain.run("Hallo, ich möchte gern über die Tiervielfalt in Südafrika reden.")


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4251.49 ms
llama_print_timings:      sample time =      47.90 ms /   168 runs   (    0.29 ms per token,  3507.38 tokens per second)
llama_print_timings: prompt eval time =   49630.96 ms /   296 tokens (  167.67 ms per token,     5.96 tokens per second)
llama_print_timings:        eval time =   39846.63 ms /   168 runs   (  237.18 ms per token,     4.22 tokens per second)
llama_print_timings:       total time =   90766.15 ms


"  Willkomm! (Welcome!) I'm here to help you learn German and answer any questions you may have. South Africa is indeed a diverse and fascinating place when it comes to wildlife, with many different species of animals that can be found there.\nCan you tell me more about what you would like to know about the animal life in South Africa? Are you interested in learning about specific animals, such as the Big Five (lion, leopard, rhinoceros, elephant, and Cape buffalo)? Or perhaps you're curious about the different ecosystems and habitats found in South Africa? Please let me know and I'll do my best to provide you with helpful and accurate information! 🦁🐴🐒"